In [23]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

# Define leg parameters
coxa_len = 28.75
tibia_len = 68.825
femur_len = 40
z_offset = 68.825
l_len = np.sqrt( z_offset**2 + coxa_len**2 ) # Z-offset can change!

def to_rads(num):
    return num*np.pi/180

def to_degs(num):
    return num/np.pi*180

In [33]:
# Forward kinematics check
leg_base = [0, 0, z_offset] # xyz
coxa_end = [0, 0, z_offset]
femur_end = [0, 0, 0]
tibia_end = [0, 0, 0]

# Input desired angles here
angles = [90.0, 104.46025455928492, 76.59469362344151]
coxa_angle = angles[0]
femur_angle = angles[1]
tibia_angle = angles[2]

femur_vert = femur_len * np.cos( to_rads(femur_angle - 90) )
femur_hor = femur_len * np.sin( to_rads(femur_angle - 90) )

theta = tibia_angle + femur_angle - 180

tibia_vert = tibia_len * np.sin( to_rads(theta) )
tibia_hor = tibia_len * np.cos( to_rads(theta) )

# calc coxa end-points
coxa_end[0] = coxa_len * np.sin( to_rads(coxa_angle-90) )
coxa_end[1] = coxa_len * np.cos( to_rads(coxa_angle-90) )

# calc femur end-points
femur_end[0] = coxa_end[0] + femur_vert * np.sin( to_rads(coxa_angle-90) )
femur_end[1] = coxa_end[1] + femur_vert * np.cos( to_rads(coxa_angle-90) )
femur_end[2] = coxa_end[2] + femur_hor * np.sin( to_rads(femur_angle-90) )

# calc tibia end-points
tibia_end[0] = femur_end[0] + tibia_vert * np.sin( to_rads(coxa_angle-90) )
tibia_end[1] = femur_end[1] + tibia_vert * np.cos( to_rads(coxa_angle-90) )
tibia_end[2] = femur_end[2] + tibia_hor * np.cos( to_rads( theta -180 ) )

print(leg_base, coxa_end, femur_end, tibia_end, theta)

# Plot both views
fig, axs = plt.subplots(2,1)
fig.suptitle("Leg IK simulation")
axs[0].set_title("Top View")
axs[0].set_xlabel("y")
axs[0].set_xlabel("x")
axs[0].plot( (leg_base[1], coxa_end[1]), (leg_base[0], coxa_end[0]), 'x-', label="Coxa")
axs[0].plot( (coxa_end[1], femur_end[1]), (coxa_end[0], femur_end[0]), 'x-', label="Femur")
axs[0].plot( (femur_end[1], tibia_end[1]), (femur_end[0], tibia_end[0]), 'x-', label="Tibia")
axs[0].grid()

axs[1].set_title("Side view")
axs[1].set_xlabel("y")
axs[1].set_xlabel("x")
axs[1].plot( (leg_base[1], coxa_end[1]), (leg_base[2], coxa_end[2]), 'x-', label="Coxa")
axs[1].plot( (coxa_end[1], femur_end[1]), (coxa_end[2], femur_end[2]), 'x-', label="Femur")
axs[1].plot( (femur_end[1], tibia_end[1]), (femur_end[2], tibia_end[2]), 'x-', label="Tibia")
axs[1].grid()

plt.legend()
plt.show()

[0, 0, 68.825] [0.0, 28.75, 68.825] [0.0, 67.48284372924779, 71.31917041614176] [0.0, 68.75, 2.5175003836609164] 1.0549481827264344


In [38]:
# inverse kinematics check (reframe)
# need to reframe the kinematics!
leg_end_coords = [0.0, 68.75, 0.0] # check back

leg_end_coords[1] -= 68.75 # remove offsets

z_offset_ik = z_offset - leg_end_coords[2]
l_len = np.sqrt( z_offset_ik**2 + femur_len**2 )
print("l_len:", l_len)

coxa_angle_ik = np.arctan2(leg_end_coords[0], leg_end_coords[1])

femur_angle_ik1 = np.arccos( z_offset_ik / l_len )
femur_angle_ik2 = np.arccos( (femur_len**2 + l_len**2 - tibia_len**2) / (2*femur_len*l_len) )
femur_angle_ik = femur_angle_ik1 + femur_angle_ik2

tibia_angle_ik = np.arccos( (femur_len**2 - l_len**2 + tibia_len**2) / (2*femur_len*tibia_len) )

angles_ik = [to_degs(coxa_angle_ik)+90, to_degs(femur_angle_ik), to_degs(tibia_angle_ik)]

print(angles_ik)

l_len: 79.60452641024881
[90.0, 90.00000000000001, 90.0]
